In [10]:
import os
import keyring
import pandas as pd

# Define server path and mount path
server_path = "smb://files/dept/InstResearch/TigerOutcomes"
mount_path = "/Volumes/TigerOutcomes"  # Where the server will be mounted on your Mac

# Retrieve credentials from Keychain
username = keyring.get_password("TigerOutcomes_Service", "username_key")
password = keyring.get_password("TigerOutcomes_Service", "password_key")

# Mount the server using AppleScript for SMB connection
os.system(f"osascript -e 'do shell script \"mount volume \\\"{server_path}\\\" as user name \\\"{username}\\\" with password \\\"{password}\\\"\"'")

# Path to the Excel file on the mounted server
file_path = f"{mount_path}/COS333_AcA_Student_Outcomes.xlsx"  # Replace 'file.xlsx' with your actual file name

# Read the Excel file with pandas if the server is successfully mounted
try:
    df = pd.read_excel(file_path)
    print(df.head())
except FileNotFoundError:
    print("File not found. Check if the server is properly mounted.")

       mount [-dfrkuvw] special | mount_point (64)o options] [-t external_type] special mount_point


               Source  RecordYear Academic_Year_Degree_Awarded Degree_Track  \
0  Academic Analytics        2021                      2013-14           AB   
1  Academic Analytics        2022                      2013-14           AB   
2  Academic Analytics        2024                      2013-14           AB   
3  Academic Analytics        2023                      2013-14           AB   
4  Academic Analytics        2021                      2009-10           AB   

       Degree_Descr Entity_Name Position UnitName  IPEDSID Country  \
0  Bachelor of Arts         NaN      NaN      NaN      NaN     NaN   
1  Bachelor of Arts         NaN      NaN      NaN      NaN     NaN   
2  Bachelor of Arts         NaN      NaN      NaN      NaN     NaN   
3  Bachelor of Arts       Kawin  Founder      NaN      NaN      US   
4  Bachelor of Arts         NaN      NaN      NaN      NaN     NaN   

    Country_Name State State_Name   StudyID  
0            NaN   NaN        NaN  COS28100  
1           